In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/EY 2025 DS Challenge'

/content/drive/MyDrive/Colab Notebooks/EY 2025 DS Challenge


# Load Original Dataset

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy.spatial import cKDTree
from xml.etree import ElementTree as ET

In [ ]:
# Load data.csv and weather.csv
data_df = pd.read_csv('datasets/data.csv', parse_dates=['datetime'])
data_df.rename(columns={'datetime': 'Datetime'}, inplace=True)
sub_df = pd.read_csv('datasets/template.csv')
footprints_df = pd.read_csv('datasets/footprints2.0.csv')

weather_df = pd.read_csv('datasets/weather.csv', parse_dates=['Datetime'])
# gdf_buildings = gpd.read_file("datasets/footprints.kml", driver='KML')

display(data_df.head())
display(sub_df.head())
display(weather_df.head())
# display(gdf_buildings.head())
display(footprints_df.head())

<ipython-input-4-c1a60d4b4c18>:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data_df = pd.read_csv('datasets/data.csv', parse_dates=['datetime'])
<ipython-input-4-c1a60d4b4c18>:7: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  weather_df = pd.read_csv('datasets/weather.csv', parse_dates=['Datetime'])


,Longitude,Latitude,Datetime,UHI Index
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634


,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,NaN
1,-73.971928,40.788875,NaN
2,-73.967080,40.789080,NaN
3,-73.972550,40.789082,NaN
4,-73.969697,40.787953,NaN


,Region,Longitude,Latitude,Datetime,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux
0,Bronx,-73.89352,40.87248,2021-07-24 06:00:00,19.3,88.2,0.8,335,12
1,Bronx,-73.89352,40.87248,2021-07-24 06:05:00,19.4,87.9,0.8,329,18
2,Bronx,-73.89352,40.87248,2021-07-24 06:10:00,19.3,87.6,0.7,321,25
3,Bronx,-73.89352,40.87248,2021-07-24 06:15:00,19.4,87.4,0.5,307,33
4,Bronx,-73.89352,40.87248,2021-07-24 06:20:00,19.4,87.0,0.2,301,42


,the_geom,BIN,CNSTRCT_YR,LSTSTATYPE,HEIGHTROOF,FEAT_CODE,GROUNDELEV,SHAPE_AREA,SHAPE_LEN,BASE_BBL,MPLUTO_BBL
0,MULTIPOLYGON (((-74.00610061881868 40.75277120...,1000000,NaN,Constructed,13.00,1000,NaN,0,0,1006740001,1006740001
1,MULTIPOLYGON (((-74.04443069358864 40.68965893...,1000002,1900.0,Constructed,119.11,2100,16.0,0,0,1000010101,1000010101
2,MULTIPOLYGON (((-74.0112906838676 40.701255197...,1000003,1900.0,Constructed,71.00,2100,-1.0,0,0,1000020002,1000020002
3,MULTIPOLYGON (((-74.01403035921098 40.70104721...,1000004,1960.0,Constructed,44.19,2100,9.0,0,0,1000030010,1000030010
4,MULTIPOLYGON (((-74.01153424206372 40.70195429...,1000005,1969.0,Constructed,662.00,2100,6.0,0,0,1000040001,1000047501


# Assign a region (Manhattan / Bronx) to train and submission data

In [ ]:
# Function to parse KML and extract county boundaries
def parse_kml(kml_file):
    tree = ET.parse(kml_file)
    root = tree.getroot()

    # Extract all polygons and their associated county names
    polygons = []
    county_names = []

    # Loop through each <Placemark> element in the KML file
    for placemark in root.iter('{http://www.opengis.net/kml/2.2}Placemark'):
        county_name = None
        polygon_coords = []

        # Extract county name (from <Data name="name"><value>...</value></Data>)
        for data in placemark.iter('{http://www.opengis.net/kml/2.2}Data'):
            if data.attrib.get('name') == 'name':
                county_name = data.find('{http://www.opengis.net/kml/2.2}value').text

        # Extract polygon coordinates
        for polygon in placemark.iter('{http://www.opengis.net/kml/2.2}Polygon'):
            for coords in polygon.iter('{http://www.opengis.net/kml/2.2}coordinates'):
                coords_text = coords.text.strip()
                for coord in coords_text.split():
                    # Try to unpack coordinates, ignore altitude if present
                    parts = coord.split(',')
                    if len(parts) >= 2:
                        lon, lat = map(float, parts[:2])
                        polygon_coords.append((lon, lat))

        # Convert coordinates into a Polygon geometry
        if polygon_coords and county_name:
            polygons.append(Polygon(polygon_coords))
            county_names.append(county_name)

    # Create a GeoDataFrame with county boundaries
    county_gdf = gpd.GeoDataFrame({
        'CountyName': county_names,
        'geometry': polygons
    }, crs="EPSG:4326")

    return county_gdf

# Function to find county for each coordinate
def get_county_for_coordinates(df, boundary_gdf):
    # Convert the Latitude and Longitude into a GeoSeries of Points
    geometry = [Point(lon, lat) for lon, lat in zip(df['Longitude'], df['Latitude'])]
    points_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

    # Initialize a list to store county names
    county_names = []

    # Check which county the point belongs to
    for _, point in points_gdf.iterrows():
        county = None
        for _, county_row in boundary_gdf.iterrows():
            if point['geometry'].within(county_row['geometry']):
                county = county_row['CountyName']  # Get the county name
                break
        county_names.append(county)

    # Add the county names as a new column to the dataframe
    df['County'] = county_names
    return df

ny_boundary = parse_kml("datasets/boundaries.kml")

# Apply the function to data_df and sub_df
data_df = get_county_for_coordinates(data_df, ny_boundary)
sub_df = get_county_for_coordinates(sub_df, ny_boundary)

In [ ]:
# Step 1: Create a new column 'Region' based on 'County' column
def assign_region(county_name):
    if county_name == 'New York County':
        return 'Manhattan'
    elif county_name == 'Bronx County':
        return 'Bronx'
    else:
        return 'Others'

# Apply the function to both dataframes
data_df['Region'] = data_df['County'].apply(assign_region)
sub_df['Region'] = sub_df['County'].apply(assign_region)

# # Step 2: Drop the 'County' column
data_df.drop('County', axis=1, inplace=True)
sub_df.drop('County', axis=1, inplace=True)

# Step 3: Count the number of "Others" regions
others_count_data = data_df['Region'].value_counts().get('Others', 0)
others_count_sub = sub_df['Region'].value_counts().get('Others', 0)

# Output the counts
print(f"Number of 'Others' regions in data_df: {others_count_data}")
print(f"Number of 'Others' regions in sub_df: {others_count_sub}")


Number of 'Others' regions in data_df: 0
Number of 'Others' regions in sub_df: 0


In [ ]:
display(data_df.head())
display(sub_df.head())

,Longitude,Latitude,Datetime,UHI Index,Region
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx


,Longitude,Latitude,UHI Index,Region
0,-73.971665,40.788763,NaN,Manhattan
1,-73.971928,40.788875,NaN,Manhattan
2,-73.967080,40.789080,NaN,Manhattan
3,-73.972550,40.789082,NaN,Manhattan
4,-73.969697,40.787953,NaN,Manhattan


# Map footprints onto orignal data and submission data
[To do: try setting different buffersize for each region]

In [ ]:
# Set a buffer size variable for radius (in meters)
# Manhattan is around 59.1km big, and bronx is around 110km big
# buffer_sizes = {
#     'Manhattan': 1500,  # Buffer size for Manhattan in meters
#     'Bronx': 1800,      # Buffer size for Bronx in meters
# }

buffer_size = 1500

In [ ]:
# Convert Longitude and Latitude columns into Point geometries
train_gdf = gpd.GeoDataFrame(data_df, geometry=gpd.points_from_xy(data_df['Longitude'], data_df['Latitude']))
sub_gdf = gpd.GeoDataFrame(sub_df, geometry=gpd.points_from_xy(sub_df['Longitude'], sub_df['Latitude']))

# Ensure all dataframes use the same CRS (Coordinate Reference System)
train_gdf.set_crs('EPSG:4326', allow_override=True, inplace=True)
sub_gdf.set_crs('EPSG:4326', allow_override=True, inplace=True)

from shapely import wkt

# Convert 'footprints_df' to GeoDataFrame with 'the_geom' as geometry
footprints_df['geometry'] = footprints_df['the_geom'].apply(wkt.loads)  # Parse WKT strings into geometries
gdf_buildings = gpd.GeoDataFrame(footprints_df, geometry='geometry')
gdf_buildings.set_crs('EPSG:4326', allow_override=True, inplace=True)

# Remove buildings constructed in 2022 or later
gdf_buildings = gdf_buildings[gdf_buildings['CNSTRCT_YR'] < 2025]


display(train_gdf.head())
display(sub_gdf.head())
display(gdf_buildings.head())

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,...,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux,traffic_volume,avg_traffic_volume,cloud_fraction,LST,NDVI,geometry
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,20.674425,10574,77.059202,59.699176,39.423162,...,47.3,2.6,165.0,621.0,29,30.0,0.103346,38.393941,0.2,POINT (-73.90917 40.81311)
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,21.354866,10559,77.076333,59.636639,39.449781,...,47.3,2.6,165.0,621.0,29,30.0,0.103346,38.393941,0.2,POINT (-73.90919 40.81304)
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,21.030104,10542,77.048473,59.569665,39.458477,...,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.785534,0.2,POINT (-73.90922 40.81298)
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,20.704158,10520,77.093251,59.518892,39.469761,...,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.785534,0.2,POINT (-73.90924 40.81291)
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,20.249984,10502,77.101124,59.485461,39.476855,...,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.358281,0.2,POINT (-73.90926 40.81284)


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,...,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux,traffic_volume,avg_traffic_volume,cloud_fraction,LST,NDVI,geometry
0,-73.971665,40.788763,NaN,Manhattan,11.815959,9331,107.151324,72.848826,94.785107,21.936281,...,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.202990,NaN,POINT (-73.97166 40.78876)
1,-73.971928,40.788875,NaN,Manhattan,17.120584,9279,107.252937,72.896173,95.171017,22.274844,...,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.202990,NaN,POINT (-73.97193 40.78888)
2,-73.967080,40.789080,NaN,Manhattan,15.469954,10074,106.589736,70.948064,87.619996,16.671932,...,45.4,3.8,202.0,349.0,63,NaN,0.153129,36.069687,NaN,POINT (-73.96708 40.78908)
3,-73.972550,40.789082,NaN,Manhattan,4.835647,9220,107.402711,72.829753,95.595381,22.765629,...,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.886594,NaN,POINT (-73.97255 40.78908)
4,-73.969697,40.787953,NaN,Manhattan,6.406210,9587,106.717221,72.386100,92.483445,20.097345,...,45.4,3.8,202.0,349.0,59,NaN,0.153129,34.500816,NaN,POINT (-73.9697 40.78795)


,the_geom,BIN,CNSTRCT_YR,LSTSTATYPE,HEIGHTROOF,FEAT_CODE,GROUNDELEV,SHAPE_AREA,SHAPE_LEN,BASE_BBL,MPLUTO_BBL,geometry
1,MULTIPOLYGON (((-74.04443069358864 40.68965893...,1000002,1900.0,Constructed,119.11,2100,16.0,0,0,1000010101,1000010101,"MULTIPOLYGON (((-74.04443 40.68966, -74.04433 ..."
2,MULTIPOLYGON (((-74.0112906838676 40.701255197...,1000003,1900.0,Constructed,71.00,2100,-1.0,0,0,1000020002,1000020002,"MULTIPOLYGON (((-74.01129 40.70126, -74.01121 ..."
3,MULTIPOLYGON (((-74.01403035921098 40.70104721...,1000004,1960.0,Constructed,44.19,2100,9.0,0,0,1000030010,1000030010,"MULTIPOLYGON (((-74.01403 40.70105, -74.01406 ..."
4,MULTIPOLYGON (((-74.01153424206372 40.70195429...,1000005,1969.0,Constructed,662.00,2100,6.0,0,0,1000040001,1000047501,"MULTIPOLYGON (((-74.01153 40.70195, -74.01152 ..."
5,MULTIPOLYGON (((-74.01092445129292 40.70246850...,1000006,1970.0,Constructed,511.00,2100,6.0,0,0,1000050001,1000057501,"MULTIPOLYGON (((-74.01092 40.70247, -74.01028 ..."


## Nearest Building Distance

In [ ]:
from sklearn.neighbors import BallTree

# Function to calculate the nearest building distance using BallTree (vectorized operations)
def nearest_building_distance_balltree(sub_gdf, gdf_buildings, buffer_size):
    # Convert building footprints and sub points into numpy arrays of coordinates (lon, lat)
    building_coords = np.array([(geom.centroid.x, geom.centroid.y) for geom in gdf_buildings.geometry])
    sub_coords = np.array([(geom.x, geom.y) for geom in sub_gdf.geometry])

    # Initialize BallTree with geospatial coordinates (convert to radians for haversine)
    tree = BallTree(np.radians(building_coords), metric='haversine')

    # Convert sub points to radians
    sub_coords_radians = np.radians(sub_coords)

    # Query BallTree to find the nearest building
    dist, _ = tree.query(sub_coords_radians, k=1)  # k=1 means nearest neighbor

    # Convert the distance from radians back to meters (radius of Earth = 6371000 meters)
    dist_in_meters = dist * 6371000

    return dist_in_meters.flatten()

# Apply the function to calculate the nearest building distance for the sub data
sub_df['nearest_building_distance'] = nearest_building_distance_balltree(sub_gdf, gdf_buildings, buffer_size)

# Apply the same to train data if necessary
data_df['nearest_building_distance'] = nearest_building_distance_balltree(train_gdf, gdf_buildings, buffer_size)

# Check the first few rows to confirm the new column
display(sub_df.head())
display(data_df.head())



,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,...,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux,traffic_volume,avg_traffic_volume,cloud_fraction,LST,NDVI
0,-73.971665,40.788763,NaN,Manhattan,11.815959,9331,107.151324,72.848826,94.785107,21.936281,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.202990,NaN
1,-73.971928,40.788875,NaN,Manhattan,17.120584,9279,107.252937,72.896173,95.171017,22.274844,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.202990,NaN
2,-73.967080,40.789080,NaN,Manhattan,15.469954,10074,106.589736,70.948064,87.619996,16.671932,...,27.3,45.4,3.8,202.0,349.0,63,NaN,0.153129,36.069687,NaN
3,-73.972550,40.789082,NaN,Manhattan,4.835647,9220,107.402711,72.829753,95.595381,22.765629,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.886594,NaN
4,-73.969697,40.787953,NaN,Manhattan,6.406210,9587,106.717221,72.386100,92.483445,20.097345,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,34.500816,NaN


,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,...,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux,traffic_volume,avg_traffic_volume,cloud_fraction,LST,NDVI
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,18.718888,10574,77.059202,59.699176,39.423162,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,38.393941,0.2
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,20.434537,10559,77.076333,59.636639,39.449781,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,38.393941,0.2
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,21.030104,10542,77.048473,59.569665,39.458477,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.785534,0.2
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,20.704158,10520,77.093251,59.518892,39.469761,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.785534,0.2
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,20.249984,10502,77.101124,59.485461,39.476855,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.358281,0.2


## Building Density

In [ ]:
# Extract the building coordinates from the centroids of the geometries
building_coords = np.array([(geom.centroid.x, geom.centroid.y) for geom in gdf_buildings.geometry])

# Initialize BallTree with geospatial coordinates (convert to radians for haversine)
tree = BallTree(np.radians(building_coords), metric='haversine')

# Function to calculate building density (number of nearby buildings within a radius)
def building_density_balltree(row, tree, r=1500):
    point = np.radians([[row['Longitude'], row['Latitude']]])  # Convert point to radians
    indices = tree.query_radius(point, r / 6371000)  # Radius in kilometers, convert to radians
    return len(indices[0])  # Return the number of nearby buildings

# Apply BallTree density calculation for data_df (based on Longitude and Latitude)
data_df['building_density'] = data_df.apply(building_density_balltree, axis=1, tree=tree)
sub_df['building_density'] = sub_df.apply(building_density_balltree, axis=1, tree=tree)


## Building Age

In [ ]:
# Define the current year
current_year = 2021

# Calculate Building Age (ignore missing values)
gdf_buildings['Building_Age'] = current_year - gdf_buildings['CNSTRCT_YR']
gdf_buildings = gdf_buildings.dropna(subset=['Building_Age'])  # Remove NaNs

# Convert building footprints to numpy array (lon, lat) and extract ages
building_coords = np.array([(geom.centroid.x, geom.centroid.y) for geom in gdf_buildings.geometry])
building_ages = gdf_buildings['Building_Age'].values  # Extract building ages

# Initialize BallTree (convert coordinates to radians for haversine distance)
tree = BallTree(np.radians(building_coords), metric='haversine')

# Function to calculate the average building age within the buffer size
def average_building_age(df, tree, buffer_size, building_ages):
    coords = np.array([(geom.x, geom.y) for geom in df.geometry])  # Extract point coordinates
    coords_radians = np.radians(coords)  # Convert to radians

    # Query buildings within the buffer (convert buffer to radians)
    indices = tree.query_radius(coords_radians, buffer_size / 6371000)

    # Compute the average age of nearby buildings
    avg_ages = [
        np.mean(building_ages[idx]) if len(idx) > 0 else np.nan  # Compute average, handle empty cases
        for idx in indices
    ]

    return avg_ages

# Apply function to calculate average building age for both datasets
sub_df['avg_building_age'] = average_building_age(sub_gdf, tree, buffer_size, building_ages)
data_df['avg_building_age'] = average_building_age(train_gdf, tree, buffer_size, building_ages)

# Check the output
display(data_df[['Longitude', 'Latitude', 'avg_building_age']].head())
display(sub_df[['Longitude', 'Latitude', 'avg_building_age']].head())


,Longitude,Latitude,avg_building_age
0,-73.909167,40.813107,76.409679
1,-73.909187,40.813045,76.425754
2,-73.909215,40.812978,76.404365
3,-73.909242,40.812908,76.447440
4,-73.909257,40.812845,76.461466


,Longitude,Latitude,avg_building_age
0,-73.971665,40.788763,106.813675
1,-73.971928,40.788875,106.924788
2,-73.967080,40.789080,106.268013
3,-73.972550,40.789082,107.083811
4,-73.969697,40.787953,106.378912


## Building Height and Elevation

In [ ]:
# Ensure numeric values for calculations
gdf_buildings['GROUNDELEV'] = pd.to_numeric(gdf_buildings['GROUNDELEV'], errors='coerce')
gdf_buildings['HEIGHTROOF'] = pd.to_numeric(gdf_buildings['HEIGHTROOF'], errors='coerce')

# Calculate Height Above Ground
gdf_buildings['Height_Above_Ground'] = gdf_buildings['HEIGHTROOF'] - gdf_buildings['GROUNDELEV']

# Remove NaN values (to avoid errors in computations)
gdf_buildings = gdf_buildings.dropna(subset=['GROUNDELEV', 'HEIGHTROOF', 'Height_Above_Ground'])

# Convert building footprints to numpy array (lon, lat)
building_coords = np.array([(geom.centroid.x, geom.centroid.y) for geom in gdf_buildings.geometry])
building_elevations = gdf_buildings['GROUNDELEV'].values  # Elevation
building_heights = gdf_buildings['HEIGHTROOF'].values  # Total Height
height_above_ground = gdf_buildings['Height_Above_Ground'].values  # Height above ground

# Initialize BallTree (convert coordinates to radians)
tree = BallTree(np.radians(building_coords), metric='haversine')

# Function to calculate average values using BallTree
def average_building_metrics(df, tree, buffer_size, values):
    coords = np.array([(geom.x, geom.y) for geom in df.geometry])
    coords_radians = np.radians(coords)

    # Query buildings within buffer
    indices = tree.query_radius(coords_radians, buffer_size / 6371000)

    # Compute average for each metricburuti
    avg_values = [
        np.mean(values[idx]) if len(idx) > 0 else np.nan  # Avoid errors when no nearby buildings
        for idx in indices
    ]
    return avg_values

# Apply function to compute average building metrics
sub_df['avg_building_elevation'] = average_building_metrics(sub_gdf, tree, buffer_size, building_elevations)
sub_df['avg_building_height'] = average_building_metrics(sub_gdf, tree, buffer_size, building_heights)
sub_df['avg_height_above_ground'] = average_building_metrics(sub_gdf, tree, buffer_size, height_above_ground)

data_df['avg_building_elevation'] = average_building_metrics(train_gdf, tree, buffer_size, building_elevations)
data_df['avg_building_height'] = average_building_metrics(train_gdf, tree, buffer_size, building_heights)
data_df['avg_height_above_ground'] = average_building_metrics(train_gdf, tree, buffer_size, height_above_ground)

# Check results
display(sub_df[['Longitude', 'Latitude', 'avg_building_elevation', 'avg_building_height', 'avg_height_above_ground']].head())
display(data_df[['Longitude', 'Latitude', 'avg_building_elevation', 'avg_building_height', 'avg_height_above_ground']].head())

,Longitude,Latitude,avg_building_elevation,avg_building_height,avg_height_above_ground
0,-73.971665,40.788763,72.826956,95.329320,22.502364
1,-73.971928,40.788875,72.876599,95.641470,22.764872
2,-73.967080,40.789080,70.930495,88.069186,17.138691
3,-73.972550,40.789082,72.799308,96.101098,23.301791
4,-73.969697,40.787953,72.361802,92.955090,20.593288


,Longitude,Latitude,avg_building_elevation,avg_building_height,avg_height_above_ground
0,-73.909167,40.813107,59.732250,39.734542,-19.997708
1,-73.909187,40.813045,59.670272,39.761382,-19.908890
2,-73.909215,40.812978,59.600471,39.763419,-19.837052
3,-73.909242,40.812908,59.550175,39.775245,-19.774930
4,-73.909257,40.812845,59.513618,39.775971,-19.737647


In [ ]:
# Optimized function to find nearest building features for all points at once
def nearest_building_height_features(df, tree, buffer_size, building_heights, height_above_ground, building_elevations):
    coords = np.array([(geom.x, geom.y) for geom in df.geometry])  # Extract point coordinates
    coords_radians = np.radians(coords)  # Convert to radians for BallTree

    # Query BallTree for buildings within the buffer (all points at once)
    indices = tree.query_radius(coords_radians, buffer_size / 6371000)  # Convert buffer to radians

    # Initialize lists to store the results
    nearest_heights = []
    nearest_heights_above_ground = []
    nearest_elevations = []

    # Get the nearest building features for each point
    for idx in indices:
        if len(idx) > 0:
            # Get the feature values of the nearest building
            nearest_heights.append(building_heights[idx[0]])  # Nearest building height
            nearest_heights_above_ground.append(height_above_ground[idx[0]])  # Nearest building height above ground
            nearest_elevations.append(building_elevations[idx[0]])  # Nearest building elevation
        else:
            # If no buildings in range, append NaN
            nearest_heights.append(np.nan)
            nearest_heights_above_ground.append(np.nan)
            nearest_elevations.append(np.nan)

    return nearest_heights, nearest_heights_above_ground, nearest_elevations

# Calculate the nearest building features for both data_df and sub_df
data_df['nearest_building_height'], data_df['nearest_building_height_above_ground'], data_df['nearest_building_elevation'] = \
    nearest_building_height_features(train_gdf, tree, buffer_size, building_heights, height_above_ground, building_elevations)

sub_df['nearest_building_height'], sub_df['nearest_building_height_above_ground'], sub_df['nearest_building_elevation'] = \
    nearest_building_height_features(sub_gdf, tree, buffer_size, building_heights, height_above_ground, building_elevations)

# Check the output for sub_df and data_df
display(data_df[['Longitude', 'Latitude', 'nearest_building_height', 'nearest_building_height_above_ground', 'nearest_building_elevation']].head())
display(sub_df[['Longitude', 'Latitude', 'nearest_building_height', 'nearest_building_height_above_ground', 'nearest_building_elevation']].head())


,Longitude,Latitude,nearest_building_height,nearest_building_height_above_ground,nearest_building_elevation
0,-73.909167,40.813107,67.0,53.0,14.0
1,-73.909187,40.813045,67.0,53.0,14.0
2,-73.909215,40.812978,67.0,53.0,14.0
3,-73.909242,40.812908,67.0,53.0,14.0
4,-73.909257,40.812845,67.0,53.0,14.0


,Longitude,Latitude,nearest_building_height,nearest_building_height_above_ground,nearest_building_elevation
0,-73.971665,40.788763,294.01,216.01,78.0
1,-73.971928,40.788875,294.01,216.01,78.0
2,-73.967080,40.789080,192.17,114.17,78.0
3,-73.972550,40.789082,294.01,216.01,78.0
4,-73.969697,40.787953,493.00,423.00,70.0


## Final Look after combining Footprints

In [ ]:
display(data_df)
display(sub_df)

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,...,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux,traffic_volume,avg_traffic_volume,cloud_fraction,LST,NDVI
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,18.718888,10662,76.409679,59.732250,39.734542,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,38.393941,0.200
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,20.434537,10647,76.425754,59.670272,39.761382,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,38.393941,0.200
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,21.030104,10629,76.404365,59.600471,39.763419,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.785534,0.200
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,20.704158,10607,76.447440,59.550175,39.775245,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.785534,0.200
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,20.249984,10588,76.461466,59.513618,39.775971,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.358281,0.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,Manhattan,135.211208,12180,101.955419,58.208364,73.181264,...,26.8,46.7,3.4,196.0,605.0,16,NaN,0.153129,30.293234,0.476
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,Manhattan,134.886861,12176,101.963042,58.221419,73.191028,...,26.8,46.7,3.4,196.0,605.0,16,NaN,0.153129,30.293234,0.476
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,Manhattan,134.753896,12168,101.964168,58.224196,73.228970,...,26.8,46.7,3.4,196.0,605.0,16,NaN,0.153129,30.440209,0.476
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,Manhattan,134.914853,12169,101.966226,58.248520,73.240985,...,27.0,46.1,2.7,209.0,620.0,16,NaN,0.153129,30.440209,0.476


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,...,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux,traffic_volume,avg_traffic_volume,cloud_fraction,LST,NDVI
0,-73.971665,40.788763,NaN,Manhattan,11.815959,9360,106.813675,72.826956,95.329320,22.502364,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.202990,NaN
1,-73.971928,40.788875,NaN,Manhattan,17.120584,9307,106.924788,72.876599,95.641470,22.764872,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.202990,NaN
2,-73.967080,40.789080,NaN,Manhattan,15.469954,10104,106.268013,70.930495,88.069186,17.138691,...,27.3,45.4,3.8,202.0,349.0,63,NaN,0.153129,36.069687,NaN
3,-73.972550,40.789082,NaN,Manhattan,4.835647,9247,107.083811,72.799308,96.101098,23.301791,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.886594,NaN
4,-73.969697,40.787953,NaN,Manhattan,6.406210,9617,106.378912,72.361802,92.955090,20.593288,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,34.500816,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,-73.919388,40.813803,NaN,Bronx,15.043305,8565,76.912668,56.716524,41.501136,-15.215388,...,27.3,44.4,3.7,162.0,511.0,29,NaN,0.103346,39.754313,0.216
1036,-73.931033,40.833178,NaN,Bronx,8.727765,8705,91.454107,71.470845,52.338347,-19.132499,...,26.9,47.7,3.5,149.0,292.0,43,51.0,0.107830,35.358739,0.272
1037,-73.934647,40.854542,NaN,Manhattan,19.228502,5350,93.230841,98.281975,55.533750,-42.748225,...,27.3,45.4,3.8,202.0,349.0,43,NaN,0.107830,39.634682,NaN
1038,-73.917223,40.815413,NaN,Bronx,20.780923,9144,76.978893,57.478799,41.349281,-16.129518,...,27.1,47.3,4.5,170.0,563.0,29,NaN,0.103346,39.200594,0.216


# Map weather data onto the original and submission data
Here we assume that the submission data is collected around the same time with the orginal data provided.

## Map the closest datetime to the submission data based on the coordinates with original data
If a coordinate is found to have similar time, it takes the closest time in 10m radius

In [ ]:
distance_threshold = 10

In [ ]:
from sklearn.neighbors import KDTree
import datetime

# Function to calculate the midpoint of two times
def calculate_midpoint_time(time1, time2):
    time1 = pd.to_datetime(time1)
    time2 = pd.to_datetime(time2)
    midpoint = time1 + (time2 - time1) / 2
    return midpoint

# Function to match closest coordinates and estimate time
def match_closest_coordinates_and_estimate_time(train_df, sub_df):
    # Convert latitude and longitude to a 2D array for KDTree
    train_coords = train_df[['Latitude', 'Longitude']].values
    sub_coords = sub_df[['Latitude', 'Longitude']].values

    # Create a KDTree from the training data
    tree = KDTree(train_coords)

    # Initialize a list to store estimated times
    estimated_times = []

    # Iterate over each sub record
    for sub_row in sub_df.itertuples():
        sub_coord = (sub_row.Latitude, sub_row.Longitude)

        # Find the closest training coordinates using KDTree
        dist, ind = tree.query([sub_coord], k=2)  # k=2 to find the two closest matches

        # Get the closest and second closest index
        closest_index = ind[0][0]
        second_closest_index = ind[0][1]

        # If both coordinates are very close, calculate the midpoint time
        if dist[0][0] < distance_threshold:  # [edit this if needed]
            time1 = train_df.loc[closest_index, 'Datetime']
            time2 = train_df.loc[second_closest_index, 'Datetime']
            estimated_time = calculate_midpoint_time(time1, time2)
        else:
            # If the closest coordinates are far enough apart, use the time from the closest match
            estimated_time = train_df.loc[closest_index, 'Datetime']

        # Append the estimated time
        estimated_times.append(estimated_time)

    # Add the estimated times as a new column in the sub DataFrame
    sub_df['Datetime'] = estimated_times

    return sub_df

# Call the function to match coordinates and estimate time
sub_df = match_closest_coordinates_and_estimate_time(data_df, sub_df)


## Based on the region and time, assign the closest weather

In [ ]:
from geopy.distance import geodesic

# Filter weather data for each region (Manhattan and Bronx)
manhattan_weather = weather_df[weather_df['Region'] == 'Manhattan']
bronx_weather = weather_df[weather_df['Region'] == 'Bronx']

# Define a function to find the closest datetime in weather data for a given row in data_df
def find_closest_weather_datetime(row, weather_df):
    data_datetime = pd.to_datetime(row['Datetime'])

    # Find the closest datetime in weather data
    weather_df['Datetime'] = pd.to_datetime(weather_df['Datetime'])
    closest_row = weather_df.iloc[(weather_df['Datetime'] - data_datetime).abs().argmin()]

    return closest_row

# Function to assign weather data to data_df based on Region and closest Datetime
def assign_weather_data(row):
    if row['Region'] == 'Manhattan':
        weather_row = find_closest_weather_datetime(row, manhattan_weather)
    elif row['Region'] == 'Bronx':
        weather_row = find_closest_weather_datetime(row, bronx_weather)
    else:
        return pd.Series([None] * 5, index=['Air Temp at Surface', 'Relative Humidity', 'Avg Wind Speed', 'Wind Direction', 'Solar Flux'])

    # Return the weather data as a new row
    return pd.Series({
        'Air Temp at Surface': weather_row['Air Temp at Surface'],
        'Relative Humidity': weather_row['Relative Humidity'],
        'Avg Wind Speed': weather_row['Avg Wind Speed'],
        'Wind Direction': weather_row['Wind Direction'],
        'Solar Flux': weather_row['Solar Flux']
    })

weather_data = data_df.apply(assign_weather_data, axis=1)
data_df = pd.concat([data_df, weather_data], axis=1)

weather_data = sub_df.apply(assign_weather_data, axis=1)
sub_df = pd.concat([sub_df, weather_data], axis=1)

## Final look on the combined weather info onto the original and submission datasets

In [ ]:
display(data_df)
display(sub_df)

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,nearest_building_height,nearest_building_height_above_ground,nearest_building_elevation,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,20.674425,10574,77.059202,59.699176,39.423162,-20.276014,13.00,1.00,12.0,27.2,47.3,2.6,165.0,621.0
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,21.354866,10559,77.076333,59.636639,39.449781,-20.186858,13.00,1.00,12.0,27.2,47.3,2.6,165.0,621.0
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,21.030104,10542,77.048473,59.569665,39.458477,-20.111188,13.00,1.00,12.0,27.2,47.3,2.6,165.0,621.0
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,20.704158,10520,77.093251,59.518892,39.469761,-20.049131,13.00,1.00,12.0,27.2,47.3,2.6,165.0,621.0
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,20.249984,10502,77.101124,59.485461,39.476855,-20.008606,13.00,1.00,12.0,27.2,47.3,2.6,165.0,621.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,Manhattan,135.211208,12138,102.313725,58.242457,72.921951,14.679494,53.13,18.13,35.0,26.8,46.7,3.4,196.0,605.0
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,Manhattan,134.886861,12134,102.321493,58.255567,72.931662,14.676096,53.13,18.13,35.0,26.8,46.7,3.4,196.0,605.0
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,Manhattan,134.753896,12126,102.322860,58.258376,72.969562,14.711186,53.13,18.13,35.0,26.8,46.7,3.4,196.0,605.0
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,Manhattan,134.914853,12127,102.324895,58.282779,72.981639,14.698860,53.13,18.13,35.0,27.0,46.1,2.7,209.0,620.0


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,nearest_building_height,nearest_building_height_above_ground,nearest_building_elevation,Datetime,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux
0,-73.971665,40.788763,NaN,Manhattan,11.815959,9331,107.151324,72.848826,94.785107,21.936281,294.010000,216.010000,78.0,2021-07-24 15:30:00,27.3,45.4,3.8,202.0,349.0
1,-73.971928,40.788875,NaN,Manhattan,17.120584,9279,107.252937,72.896173,95.171017,22.274844,152.520000,74.520000,78.0,2021-07-24 15:30:00,27.3,45.4,3.8,202.0,349.0
2,-73.967080,40.789080,NaN,Manhattan,15.469954,10074,106.589736,70.948064,87.619996,16.671932,192.170000,114.170000,78.0,2021-07-24 15:29:00,27.3,45.4,3.8,202.0,349.0
3,-73.972550,40.789082,NaN,Manhattan,4.835647,9220,107.402711,72.829753,95.595381,22.765629,156.950000,76.950000,80.0,2021-07-24 15:30:00,27.3,45.4,3.8,202.0,349.0
4,-73.969697,40.787953,NaN,Manhattan,6.406210,9587,106.717221,72.386100,92.483445,20.097345,452.780000,387.780000,65.0,2021-07-24 15:29:00,27.3,45.4,3.8,202.0,349.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,-73.919388,40.813803,NaN,Bronx,15.043305,8476,77.737966,56.781139,40.878732,-15.902407,16.230000,4.230000,12.0,2021-07-24 15:26:00,27.3,44.4,3.7,162.0,511.0
1036,-73.931033,40.833178,NaN,Bronx,8.727765,8633,92.231669,71.663458,51.869178,-19.794280,59.569627,37.569627,22.0,2021-07-24 15:35:00,26.9,47.7,3.5,149.0,292.0
1037,-73.934647,40.854542,NaN,Manhattan,19.228502,5307,94.000754,98.614892,55.035343,-43.579548,13.980000,0.980000,13.0,2021-07-24 15:32:00,27.3,45.4,3.8,202.0,349.0
1038,-73.917223,40.815413,NaN,Bronx,20.780923,9050,77.796243,57.534536,40.803620,-16.730916,13.000000,1.000000,12.0,2021-07-24 15:28:00,27.1,47.3,4.5,170.0,563.0


# Combine traffic volume data


## Load the Traffic Data

In [ ]:
traffic_df = pd.read_csv('datasets/traffic_volume.csv')
traffic_df.rename(columns={'Yr': 'year', 'M': 'month', 'D': 'day', 'HH': 'hour', 'MM': 'minute'}, inplace=True)
traffic_df['Datetime'] = pd.to_datetime(traffic_df[['year', 'month', 'day', 'hour', 'minute']])
display(traffic_df)
traffic_df.columns

,RequestID,Boro,year,month,day,hour,minute,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction,Datetime
0,34023,Manhattan,2021,7,12,15,0,63,38484,POINT (996446.8857448514 233920.51304525515),MORNINGSIDE AVENUE,West 120 Street,West 121 Street,NB,2021-07-12 15:00:00
1,34023,Manhattan,2021,7,12,15,15,57,38484,POINT (996446.8857448514 233920.51304525515),MORNINGSIDE AVENUE,West 120 Street,West 121 Street,NB,2021-07-12 15:15:00
2,34023,Manhattan,2021,7,12,15,30,71,38484,POINT (996446.8857448514 233920.51304525515),MORNINGSIDE AVENUE,West 120 Street,West 121 Street,NB,2021-07-12 15:30:00
3,34023,Manhattan,2021,7,12,15,45,69,38484,POINT (996446.8857448514 233920.51304525515),MORNINGSIDE AVENUE,West 120 Street,West 121 Street,NB,2021-07-12 15:45:00
4,34023,Manhattan,2021,7,12,16,0,75,38484,POINT (996446.8857448514 233920.51304525515),MORNINGSIDE AVENUE,West 120 Street,West 121 Street,NB,2021-07-12 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,33539,Manhattan,2021,7,19,15,45,65,229580,POINT (988560.7694578806 212251.00973740162),WEST 35 STREET,5 Avenue,Dead end,WB,2021-07-19 15:45:00
380,33539,Manhattan,2021,7,19,16,0,76,229580,POINT (988560.7694578806 212251.00973740162),WEST 35 STREET,5 Avenue,Dead end,WB,2021-07-19 16:00:00
381,33539,Manhattan,2021,7,19,16,15,85,229580,POINT (988560.7694578806 212251.00973740162),WEST 35 STREET,5 Avenue,Dead end,WB,2021-07-19 16:15:00
382,33539,Manhattan,2021,7,19,16,30,87,229580,POINT (988560.7694578806 212251.00973740162),WEST 35 STREET,5 Avenue,Dead end,WB,2021-07-19 16:30:00


Index(['RequestID', 'Boro', 'year', 'month', 'day', 'hour', 'minute', 'Vol',
       'SegmentID', 'WktGeom', 'street', 'fromSt', 'toSt', 'Direction',
       'Datetime'],
      dtype='object')

## Convert the traffic data, train and submission data into the same CRS

In [ ]:
from shapely import wkt
from datetime import datetime

# Convert the traffic data geometry (WKT) to geopandas geometry if needed
# If the traffic data already has geometries, skip this step.
traffic_df['geometry'] = traffic_df['WktGeom'].apply(wkt.loads)
traffic_gdf = gpd.GeoDataFrame(traffic_df, geometry='geometry')

# Set the CRS (Coordinate Reference System) for traffic data (assuming State Plane)
traffic_gdf.set_crs('EPSG:2263', inplace=True)  # Example CRS (State Plane)

# Convert the data_df (Longitude, Latitude) to GeoDataFrame (using EPSG:4326 for WGS84)
data_gdf = gpd.GeoDataFrame(data_df, geometry=gpd.points_from_xy(data_df['Longitude'], data_df['Latitude']))
data_gdf.set_crs('EPSG:4326', inplace=True)
sub_gdf = gpd.GeoDataFrame(sub_df, geometry=gpd.points_from_xy(sub_df['Longitude'], sub_df['Latitude']))
sub_gdf.set_crs('EPSG:4326', inplace=True)

# Reproject data_gdf and sub_gdf to the same CRS as traffic_gdf (State Plane)
data_gdf = data_gdf.to_crs('EPSG:2263')
sub_gdf = sub_gdf.to_crs('EPSG:2263')


In [ ]:

# Extract the coordinates from the traffic data
traffic_coords = np.array([(geom.x, geom.y) for geom in traffic_gdf.geometry])

# Initialize the BallTree for nearest neighbor search
tree = BallTree(traffic_coords)

# Function to find the nearest traffic volume for each point in data_gdf
def find_nearest_traffic(row, tree, traffic_gdf, region):
    # Convert the row's point to the same CRS as traffic data (State Plane)
    point = np.array([[row.geometry.x, row.geometry.y]])

    # Query the BallTree for the nearest traffic record
    dist, indices = tree.query(point, k=1)  # k=1 for nearest neighbor

    # Extract the nearest traffic data
    nearest_traffic = traffic_gdf.iloc[indices[0][0]]

    # Set the reference date based on the region
    if region == 'Manhattan':
        target_date = datetime(2021, 7, 17)  # Use July 17, 2021 for Manhattan
    elif region == 'Bronx':
        target_date = datetime(2021, 7, 31)  # Use July 31, 2021 for Bronx

    # Get the closest date in traffic data
    nearest_traffic_time = nearest_traffic['Datetime']

    # Calculate the time difference (in seconds) between the nearest traffic record and the target date
    time_diff = abs(nearest_traffic_time - target_date).total_seconds() / 3600  # Convert to hours

    return nearest_traffic['Vol'], time_diff

# Apply the function to each row in data_gdf
data_df['traffic_volume'], data_df['time_diff'] = zip(*data_gdf.apply(
    lambda row: find_nearest_traffic(row, tree, traffic_gdf, region=row['Region']), axis=1))
sub_df['traffic_volume'], sub_df['time_diff'] = zip(*sub_gdf.apply(
    lambda row: find_nearest_traffic(row, tree, traffic_gdf, region=row['Region']), axis=1))

data_df = data_df.drop(columns=['time_diff'])
sub_df = sub_df.drop(columns=['time_diff'])

In [ ]:
# Set reference dates for each region
reference_dates = {
    'Manhattan': datetime(2021, 7, 17),
    'Bronx': datetime(2021, 7, 31)
}

# Function to calculate average traffic volume using BallTree within buffer
def average_traffic_volume(row, tree, traffic_gdf, region, buffer_size):
    if region not in reference_dates:
        return np.nan  # If region is not found, return NaN

    target_date = reference_dates[region]

    # Convert the row's coordinate to an array
    point = np.array([[row.geometry.x, row.geometry.y]])

    # Find all traffic points within buffer_size using BallTree
    indices = tree.query_radius(point, r=buffer_size)  # Find all points within the radius

    if len(indices[0]) == 0:
        return np.nan  # No nearby traffic data found

    # Extract the nearby traffic data
    nearby_traffic = traffic_gdf.iloc[indices[0]]

    # Filter by closest date to target_date
    nearby_traffic['time_diff'] = (nearby_traffic['Datetime'] - target_date).abs()
    min_time_diff = nearby_traffic['time_diff'].min()
    nearby_traffic = nearby_traffic[nearby_traffic['time_diff'] == min_time_diff]

    # Compute the average traffic volume
    avg_vol = nearby_traffic['Vol'].mean() if not nearby_traffic.empty else np.nan

    return avg_vol

# Apply the function to data_gdf and sub_gdf
data_df['avg_traffic_volume'] = data_gdf.apply(
    lambda row: average_traffic_volume(row, tree, traffic_gdf, row['Region'], buffer_size), axis=1
)
sub_df['avg_traffic_volume'] = sub_gdf.apply(
    lambda row: average_traffic_volume(row, tree, traffic_gdf, row['Region'], buffer_size), axis=1
)

In [ ]:
# Display sample results
display(data_df)
display(sub_df)

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,...,nearest_building_height,nearest_building_height_above_ground,nearest_building_elevation,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux,traffic_volume,avg_traffic_volume
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,20.674425,10574,77.059202,59.699176,39.423162,...,13.00,1.00,12.0,27.2,47.3,2.6,165.0,621.0,29,30.0
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,21.354866,10559,77.076333,59.636639,39.449781,...,13.00,1.00,12.0,27.2,47.3,2.6,165.0,621.0,29,30.0
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,21.030104,10542,77.048473,59.569665,39.458477,...,13.00,1.00,12.0,27.2,47.3,2.6,165.0,621.0,29,30.0
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,20.704158,10520,77.093251,59.518892,39.469761,...,13.00,1.00,12.0,27.2,47.3,2.6,165.0,621.0,29,30.0
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,20.249984,10502,77.101124,59.485461,39.476855,...,13.00,1.00,12.0,27.2,47.3,2.6,165.0,621.0,29,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,Manhattan,135.211208,12138,102.313725,58.242457,72.921951,...,53.13,18.13,35.0,26.8,46.7,3.4,196.0,605.0,16,NaN
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,Manhattan,134.886861,12134,102.321493,58.255567,72.931662,...,53.13,18.13,35.0,26.8,46.7,3.4,196.0,605.0,16,NaN
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,Manhattan,134.753896,12126,102.322860,58.258376,72.969562,...,53.13,18.13,35.0,26.8,46.7,3.4,196.0,605.0,16,NaN
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,Manhattan,134.914853,12127,102.324895,58.282779,72.981639,...,53.13,18.13,35.0,27.0,46.1,2.7,209.0,620.0,16,NaN


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,...,nearest_building_height_above_ground,nearest_building_elevation,Datetime,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux,traffic_volume,avg_traffic_volume
0,-73.971665,40.788763,NaN,Manhattan,11.815959,9331,107.151324,72.848826,94.785107,21.936281,...,216.010000,78.0,2021-07-24 15:30:00,27.3,45.4,3.8,202.0,349.0,59,NaN
1,-73.971928,40.788875,NaN,Manhattan,17.120584,9279,107.252937,72.896173,95.171017,22.274844,...,74.520000,78.0,2021-07-24 15:30:00,27.3,45.4,3.8,202.0,349.0,59,NaN
2,-73.967080,40.789080,NaN,Manhattan,15.469954,10074,106.589736,70.948064,87.619996,16.671932,...,114.170000,78.0,2021-07-24 15:29:00,27.3,45.4,3.8,202.0,349.0,63,NaN
3,-73.972550,40.789082,NaN,Manhattan,4.835647,9220,107.402711,72.829753,95.595381,22.765629,...,76.950000,80.0,2021-07-24 15:30:00,27.3,45.4,3.8,202.0,349.0,59,NaN
4,-73.969697,40.787953,NaN,Manhattan,6.406210,9587,106.717221,72.386100,92.483445,20.097345,...,387.780000,65.0,2021-07-24 15:29:00,27.3,45.4,3.8,202.0,349.0,59,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,-73.919388,40.813803,NaN,Bronx,15.043305,8476,77.737966,56.781139,40.878732,-15.902407,...,4.230000,12.0,2021-07-24 15:26:00,27.3,44.4,3.7,162.0,511.0,29,NaN
1036,-73.931033,40.833178,NaN,Bronx,8.727765,8633,92.231669,71.663458,51.869178,-19.794280,...,37.569627,22.0,2021-07-24 15:35:00,26.9,47.7,3.5,149.0,292.0,43,51.0
1037,-73.934647,40.854542,NaN,Manhattan,19.228502,5307,94.000754,98.614892,55.035343,-43.579548,...,0.980000,13.0,2021-07-24 15:32:00,27.3,45.4,3.8,202.0,349.0,43,NaN
1038,-73.917223,40.815413,NaN,Bronx,20.780923,9050,77.796243,57.534536,40.803620,-16.730916,...,1.000000,12.0,2021-07-24 15:28:00,27.1,47.3,4.5,170.0,563.0,29,NaN


# Combine Chen's dataset (NDVI, LST, Cloud Fraction for train and submission data)


In [ ]:
# Cloud fraction
cloud_train = pd.read_csv('datasets/cloud_train.csv')
cloud_val = pd.read_csv('datasets/cloud_val_data.csv')

# LST
lst = pd.read_csv('datasets/lst_data.csv')
val_lst = pd.read_csv('datasets/val_lst_data.csv')

# NDVI
NDVI = pd.read_csv('datasets/NDVI_df.csv')
NDVI_val = pd.read_csv('datasets/val_NDVI.csv')

display(cloud_train)
display(cloud_val)
display(lst)
display(val_lst)
display(NDVI)
display(NDVI_val)

,Unnamed: 0,cloud_fraction
0,0,0.103346
1,1,0.103346
2,2,0.103346
3,3,0.103346
4,4,0.103346
...,...,...
11224,11224,0.153129
11225,11225,0.153129
11226,11226,0.153129
11227,11227,0.153129


,Unnamed: 0,cloud_fraction
0,0,0.153129
1,1,0.153129
2,2,0.153129
3,3,0.153129
4,4,0.153129
...,...,...
1035,1035,0.103346
1036,1036,0.107830
1037,1037,0.107830
1038,1038,0.103346


,LST
0,38.393941
1,38.393941
2,37.785534
3,37.785534
4,37.358281
...,...
11224,30.293234
11225,30.293234
11226,30.440209
11227,30.440209


,LST
0,36.202990
1,36.202990
2,36.069687
3,36.886594
4,34.500816
...,...
1035,39.754313
1036,35.358739
1037,39.634682
1038,39.200594


,Unnamed: 0,NDVI
0,0,0.200
1,1,0.200
2,2,0.200
3,3,0.200
4,4,0.200
...,...,...
11224,11224,0.476
11225,11225,0.476
11226,11226,0.476
11227,11227,0.476


,Unnamed: 0,AOD_440,AOD_550,AOD_670,AOD_865,AOD_2250,syn-oa01,syn-oa02,syn-oa03,syn-oa04,...,syn-oa10,syn-oa11,syn-oa12,syn-oa16,syn-oa17,syn-oa18,syn-oa21,water_vapour,ozone,NDVI
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,1035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.216
1036,1036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.272
1037,1037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1038,1038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.216


In [ ]:
data_df = pd.concat([data_df, cloud_train['cloud_fraction'], lst['LST'], NDVI['NDVI']], axis=1)
sub_df = pd.concat([sub_df, cloud_val['cloud_fraction'], val_lst['LST'], NDVI_val['NDVI']], axis=1)

display(data_df)
display(sub_df)

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,...,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux,traffic_volume,avg_traffic_volume,cloud_fraction,LST,NDVI
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,20.674425,10574,77.059202,59.699176,39.423162,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,38.393941,0.200
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,21.354866,10559,77.076333,59.636639,39.449781,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,38.393941,0.200
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,21.030104,10542,77.048473,59.569665,39.458477,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.785534,0.200
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,20.704158,10520,77.093251,59.518892,39.469761,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.785534,0.200
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,20.249984,10502,77.101124,59.485461,39.476855,...,27.2,47.3,2.6,165.0,621.0,29,30.0,0.103346,37.358281,0.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,Manhattan,135.211208,12138,102.313725,58.242457,72.921951,...,26.8,46.7,3.4,196.0,605.0,16,NaN,0.153129,30.293234,0.476
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,Manhattan,134.886861,12134,102.321493,58.255567,72.931662,...,26.8,46.7,3.4,196.0,605.0,16,NaN,0.153129,30.293234,0.476
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,Manhattan,134.753896,12126,102.322860,58.258376,72.969562,...,26.8,46.7,3.4,196.0,605.0,16,NaN,0.153129,30.440209,0.476
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,Manhattan,134.914853,12127,102.324895,58.282779,72.981639,...,27.0,46.1,2.7,209.0,620.0,16,NaN,0.153129,30.440209,0.476


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,...,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux,traffic_volume,avg_traffic_volume,cloud_fraction,LST,NDVI
0,-73.971665,40.788763,NaN,Manhattan,11.815959,9331,107.151324,72.848826,94.785107,21.936281,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.202990,NaN
1,-73.971928,40.788875,NaN,Manhattan,17.120584,9279,107.252937,72.896173,95.171017,22.274844,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.202990,NaN
2,-73.967080,40.789080,NaN,Manhattan,15.469954,10074,106.589736,70.948064,87.619996,16.671932,...,27.3,45.4,3.8,202.0,349.0,63,NaN,0.153129,36.069687,NaN
3,-73.972550,40.789082,NaN,Manhattan,4.835647,9220,107.402711,72.829753,95.595381,22.765629,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,36.886594,NaN
4,-73.969697,40.787953,NaN,Manhattan,6.406210,9587,106.717221,72.386100,92.483445,20.097345,...,27.3,45.4,3.8,202.0,349.0,59,NaN,0.153129,34.500816,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,-73.919388,40.813803,NaN,Bronx,15.043305,8476,77.737966,56.781139,40.878732,-15.902407,...,27.3,44.4,3.7,162.0,511.0,29,NaN,0.103346,39.754313,0.216
1036,-73.931033,40.833178,NaN,Bronx,8.727765,8633,92.231669,71.663458,51.869178,-19.794280,...,26.9,47.7,3.5,149.0,292.0,43,51.0,0.107830,35.358739,0.272
1037,-73.934647,40.854542,NaN,Manhattan,19.228502,5307,94.000754,98.614892,55.035343,-43.579548,...,27.3,45.4,3.8,202.0,349.0,43,NaN,0.107830,39.634682,NaN
1038,-73.917223,40.815413,NaN,Bronx,20.780923,9050,77.796243,57.534536,40.803620,-16.730916,...,27.1,47.3,4.5,170.0,563.0,29,NaN,0.103346,39.200594,0.216


# Combine Sentinel2 Information

In [ ]:
!pip install planetary-computer rioxarray

In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
import geopandas as gpd

# Geospatial operations
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp
from rasterio.warp import transform_bounds
from rasterio.windows import from_bounds

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm

## Load S2 Tiff

In [ ]:
# Open the GeoTIFF file
S2_TIFF = "datasets/S2_sample_all.tiff"

# Read the bands from the GeoTIFF file
with rasterio.open(S2_TIFF) as src1:
    sband1 = src1.read(1)  # Band [B01]
    sband2 = src1.read(2)  # Band [B02]
    sband3 = src1.read(3)  # Band [B03]
    sband4 = src1.read(4)  # Band [B04]
    sband5 = src1.read(5)  # Band [B05]
    sband6 = src1.read(6)  # Band [B06]
    sband7 = src1.read(7)  # Band [B07]
    sband8 = src1.read(8)  # Band [B08]
    sband8a = src1.read(9)  # Band [B8a]
    sband11 = src1.read(10)  # Band [B11]
    sband12 = src1.read(11)  # Band [B12]

## Extract S2 Info

In [ ]:
# Extracts satellite band values from a GeoTIFF based on coordinates from a csv file and returns them in a DataFrame.

def map_satellite_data(tiff_path, csv_path):

    # Load the GeoTIFF data
    data = rxr.open_rasterio(tiff_path)
    tiff_crs = data.rio.crs

    # Read the Excel file using pandas
    df = pd.read_csv(csv_path)
    latitudes = df['Latitude'].values
    longitudes = df['Longitude'].values

    # 3. Convert lat/long to the GeoTIFF's CRS
    # Create a Proj object for EPSG:4326 (WGS84 - lat/long) and the GeoTIFF's CRS
    proj_wgs84 = Proj(init='epsg:4326')  # EPSG:4326 is the common lat/long CRS
    proj_tiff = Proj(tiff_crs)

    # Create a transformer object
    transformer = Transformer.from_proj(proj_wgs84, proj_tiff)

    B01_values = []
    B02_values = []
    B03_values = []
    B04_values = []
    B05_values = []
    B06_values = []
    B07_values = []
    B08_values = []
    B8A_values = []
    B11_values = []
    B12_values = []

# Iterate over the latitudes and longitudes, and extract the corresponding band values
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping values"):
    # Assuming the correct dimensions are 'y' and 'x' (replace these with actual names from data.coords)

        B01_value = data.sel(x=lon, y=lat, band=1, method="nearest").values
        B01_values.append(B01_value)

        B02_value = data.sel(x=lon, y=lat, band=2, method="nearest").values
        B02_values.append(B02_value)

        B03_value = data.sel(x=lon, y=lat, band=1, method="nearest").values
        B03_values.append(B03_value)

        B04_value = data.sel(x=lon, y=lat, band=2, method="nearest").values
        B04_values.append(B04_value)

        B05_value = data.sel(x=lon, y=lat, band=1, method="nearest").values
        B05_values.append(B05_value)

        B06_value = data.sel(x=lon, y=lat, band=3, method="nearest").values
        B06_values.append(B06_value)

        B07_value = data.sel(x=lon, y=lat, band=3, method="nearest").values
        B07_values.append(B07_value)

        B08_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B08_values.append(B08_value)

        B8A_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B8A_values.append(B8A_value)

        B11_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B11_values.append(B11_value)

        B12_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B12_values.append(B12_value)

    # Create a DataFrame with the band values
    # Create a DataFrame to store the band values
    df = pd.DataFrame()
    df['SB01'] = B01_values
    df['SB02'] = B02_values
    df['SB03'] = B03_values
    df['SB04'] = B04_values
    df['SB06'] = B06_values
    df['SB07'] = B07_values
    df['SB08'] = B08_values
    df['SB8A'] = B8A_values
    df['SB11'] = B11_values
    df['SB12'] = B12_values

    return df


In [ ]:
# Mapping satellite data with training data.
S2_data = map_satellite_data('datasets/S2_sample_all.tiff', 'datasets/data.csv')
S2_sub = map_satellite_data('datasets/S2_sample_all.tiff', 'datasets/template.csv')

Mapping values: 100%|██████████| 1040/1040 [00:18<00:00, 56.55it/s]


In [ ]:
S2_data['SNDVI'] = (S2_data['SB08'] - S2_data['SB04']) / (S2_data['SB08'] + S2_data['SB04'])
S2_data['SNDVI'] = S2_data['SNDVI'].replace([np.inf, -np.inf], np.nan)

S2_sub['SNDVI'] = (S2_sub['SB08'] - S2_sub['SB04']) / (S2_sub['SB08'] + S2_sub['SB04'])

In [ ]:
display(S2_data.head())
display(S2_sub.head())

,SB01,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12,SNDVI
0,846.0,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0,-0.002887
1,846.0,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0,-0.002887
2,846.0,583.0,846.0,583.0,818.0,818.0,709.0,709.0,709.0,709.0,0.097523
3,846.0,581.0,846.0,581.0,733.0,733.0,657.0,657.0,657.0,657.0,0.061389
4,846.0,655.0,846.0,655.0,744.0,744.0,745.0,745.0,745.0,745.0,0.064286


,SB01,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12,SNDVI
0,811.0,459.0,811.0,459.0,617.0,617.0,432.0,432.0,432.0,432.0,-0.030303
1,1208.0,562.0,1208.0,562.0,731.0,731.0,647.0,647.0,647.0,647.0,0.070306
2,899.0,955.0,899.0,955.0,1052.0,1052.0,1188.0,1188.0,1188.0,1188.0,0.108726
3,1193.0,1132.0,1193.0,1132.0,1364.0,1364.0,1512.0,1512.0,1512.0,1512.0,0.143722
4,1097.0,1506.0,1097.0,1506.0,1642.0,1642.0,1688.0,1688.0,1688.0,1688.0,0.056982


In [ ]:
# Combining ground data and final data into a single dataset.
uhi_data = pd.concat([data_df,S2_data], axis=1)
display(uhi_data)

uhi_sub = pd.concat([sub_df,S2_sub], axis=1)
display(uhi_sub)

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,...,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12,SNDVI
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,18.718888,10662,76.409679,59.732250,39.734542,...,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0,-0.002887
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,20.434537,10647,76.425754,59.670272,39.761382,...,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0,-0.002887
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,21.030104,10629,76.404365,59.600471,39.763419,...,583.0,846.0,583.0,818.0,818.0,709.0,709.0,709.0,709.0,0.097523
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,20.704158,10607,76.447440,59.550175,39.775245,...,581.0,846.0,581.0,733.0,733.0,657.0,657.0,657.0,657.0,0.061389
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,20.249984,10588,76.461466,59.513618,39.775971,...,655.0,846.0,655.0,744.0,744.0,745.0,745.0,745.0,745.0,0.064286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,Manhattan,135.211208,12180,101.955419,58.208364,73.181264,...,473.0,481.0,473.0,708.0,708.0,528.0,528.0,528.0,528.0,0.054945
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,Manhattan,134.886861,12176,101.963042,58.221419,73.191028,...,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0,0.060870
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,Manhattan,134.753896,12168,101.964168,58.224196,73.228970,...,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0,0.060870
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,Manhattan,134.914853,12169,101.966226,58.248520,73.240985,...,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0,0.060870


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,...,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12,SNDVI
0,-73.971665,40.788763,NaN,Manhattan,11.815959,9360,106.813675,72.826956,95.329320,22.502364,...,459.0,811.0,459.0,617.0,617.0,432.0,432.0,432.0,432.0,-0.030303
1,-73.971928,40.788875,NaN,Manhattan,17.120584,9307,106.924788,72.876599,95.641470,22.764872,...,562.0,1208.0,562.0,731.0,731.0,647.0,647.0,647.0,647.0,0.070306
2,-73.967080,40.789080,NaN,Manhattan,15.469954,10104,106.268013,70.930495,88.069186,17.138691,...,955.0,899.0,955.0,1052.0,1052.0,1188.0,1188.0,1188.0,1188.0,0.108726
3,-73.972550,40.789082,NaN,Manhattan,4.835647,9247,107.083811,72.799308,96.101098,23.301791,...,1132.0,1193.0,1132.0,1364.0,1364.0,1512.0,1512.0,1512.0,1512.0,0.143722
4,-73.969697,40.787953,NaN,Manhattan,6.406210,9617,106.378912,72.361802,92.955090,20.593288,...,1506.0,1097.0,1506.0,1642.0,1642.0,1688.0,1688.0,1688.0,1688.0,0.056982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,-73.919388,40.813803,NaN,Bronx,15.043305,8565,76.912668,56.716524,41.501136,-15.215388,...,1086.0,1474.0,1086.0,1382.0,1382.0,1474.0,1474.0,1474.0,1474.0,0.151562
1036,-73.931033,40.833178,NaN,Bronx,8.727765,8705,91.454107,71.470845,52.338347,-19.132499,...,548.0,1014.0,548.0,766.0,766.0,797.0,797.0,797.0,797.0,0.18513
1037,-73.934647,40.854542,NaN,Manhattan,19.228502,5350,93.230841,98.281975,55.533750,-42.748225,...,1184.0,917.0,1184.0,1462.0,1462.0,1538.0,1538.0,1538.0,1538.0,0.130051
1038,-73.917223,40.815413,NaN,Bronx,20.780923,9144,76.978893,57.478799,41.349281,-16.129518,...,1066.0,1890.0,1066.0,1244.0,1244.0,1368.0,1368.0,1368.0,1368.0,0.124076


# Save final combined datasets

In [ ]:
print(uhi_data.columns)
print(uhi_sub.columns)

Index(['Longitude', 'Latitude', 'Datetime', 'UHI Index', 'Region',
       'nearest_building_distance', 'building_density', 'avg_building_age',
       'avg_building_elevation', 'avg_building_height',
       'avg_height_above_ground', 'nearest_building_height',
       'nearest_building_height_above_ground', 'nearest_building_elevation',
       'Air Temp at Surface', 'Relative Humidity', 'Avg Wind Speed',
       'Wind Direction', 'Solar Flux', 'traffic_volume', 'avg_traffic_volume',
       'cloud_fraction', 'LST', 'NDVI', 'SB01', 'SB02', 'SB03', 'SB04', 'SB06',
       'SB07', 'SB08', 'SB8A', 'SB11', 'SB12', 'SNDVI'],
      dtype='object')
Index(['Longitude', 'Latitude', 'UHI Index', 'Region',
       'nearest_building_distance', 'building_density', 'avg_building_age',
       'avg_building_elevation', 'avg_building_height',
       'avg_height_above_ground', 'nearest_building_height',
       'nearest_building_height_above_ground', 'nearest_building_elevation',
       'Datetime', 'Air Temp at

In [ ]:
display(uhi_data)
display(uhi_sub)

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,...,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12,SNDVI
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,18.718888,10662,76.409679,59.732250,39.734542,...,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0,-0.002887
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,20.434537,10647,76.425754,59.670272,39.761382,...,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0,-0.002887
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,21.030104,10629,76.404365,59.600471,39.763419,...,583.0,846.0,583.0,818.0,818.0,709.0,709.0,709.0,709.0,0.097523
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,20.704158,10607,76.447440,59.550175,39.775245,...,581.0,846.0,581.0,733.0,733.0,657.0,657.0,657.0,657.0,0.061389
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,20.249984,10588,76.461466,59.513618,39.775971,...,655.0,846.0,655.0,744.0,744.0,745.0,745.0,745.0,745.0,0.064286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,Manhattan,135.211208,12180,101.955419,58.208364,73.181264,...,473.0,481.0,473.0,708.0,708.0,528.0,528.0,528.0,528.0,0.054945
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,Manhattan,134.886861,12176,101.963042,58.221419,73.191028,...,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0,0.060870
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,Manhattan,134.753896,12168,101.964168,58.224196,73.228970,...,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0,0.060870
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,Manhattan,134.914853,12169,101.966226,58.248520,73.240985,...,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0,0.060870


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,...,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12,SNDVI
0,-73.971665,40.788763,NaN,Manhattan,11.815959,9360,106.813675,72.826956,95.329320,22.502364,...,459.0,811.0,459.0,617.0,617.0,432.0,432.0,432.0,432.0,-0.030303
1,-73.971928,40.788875,NaN,Manhattan,17.120584,9307,106.924788,72.876599,95.641470,22.764872,...,562.0,1208.0,562.0,731.0,731.0,647.0,647.0,647.0,647.0,0.070306
2,-73.967080,40.789080,NaN,Manhattan,15.469954,10104,106.268013,70.930495,88.069186,17.138691,...,955.0,899.0,955.0,1052.0,1052.0,1188.0,1188.0,1188.0,1188.0,0.108726
3,-73.972550,40.789082,NaN,Manhattan,4.835647,9247,107.083811,72.799308,96.101098,23.301791,...,1132.0,1193.0,1132.0,1364.0,1364.0,1512.0,1512.0,1512.0,1512.0,0.143722
4,-73.969697,40.787953,NaN,Manhattan,6.406210,9617,106.378912,72.361802,92.955090,20.593288,...,1506.0,1097.0,1506.0,1642.0,1642.0,1688.0,1688.0,1688.0,1688.0,0.056982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,-73.919388,40.813803,NaN,Bronx,15.043305,8565,76.912668,56.716524,41.501136,-15.215388,...,1086.0,1474.0,1086.0,1382.0,1382.0,1474.0,1474.0,1474.0,1474.0,0.151562
1036,-73.931033,40.833178,NaN,Bronx,8.727765,8705,91.454107,71.470845,52.338347,-19.132499,...,548.0,1014.0,548.0,766.0,766.0,797.0,797.0,797.0,797.0,0.18513
1037,-73.934647,40.854542,NaN,Manhattan,19.228502,5350,93.230841,98.281975,55.533750,-42.748225,...,1184.0,917.0,1184.0,1462.0,1462.0,1538.0,1538.0,1538.0,1538.0,0.130051
1038,-73.917223,40.815413,NaN,Bronx,20.780923,9144,76.978893,57.478799,41.349281,-16.129518,...,1066.0,1890.0,1066.0,1244.0,1244.0,1368.0,1368.0,1368.0,1368.0,0.124076


In [ ]:
uhi_data.to_csv('data_s2.csv', index=False)
uhi_sub.to_csv('sub_s2.csv', index=False)